In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font',family='Malgun Gothic')
import seaborn as sns
import pandas as pd
import scipy.stats as stats
import numpy as np

In [2]:
df_BM = pd.read_csv('../../datasets/JoinCollection NSC2_BND_M20.csv')
df_BM ## 고혈압환자들만 select

,Unnamed: 0,EDC_SBA,BTH_YYYY,DTH_YYYYMM,MCARE_SUBJ_CD,OPRTN_YN,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,STD_YYYY
0,0,3000,1921LE,200805.0,1,0,30,5.0,2002
1,1,6000,1921LE,200805.0,1,0,60,5.0,2002
2,2,3000,1921LE,200805.0,1,0,30,5.0,2002
3,3,4200,1921LE,200805.0,23,0,7,1.0,2007
4,4,1500,1921LE,201409.0,1,0,7,5.0,2004
...,...,...,...,...,...,...,...,...,...
4624,4624,3000,2005,NaN,1,0,30,1.0,2005
4625,4625,3000,2005,NaN,1,0,30,1.0,2005
4626,4626,3000,2005,NaN,1,0,30,1.0,2005
4627,4627,3000,2005,NaN,1,0,30,1.0,2006


### 심결본인부담금-영향이 있을 것으로 예상되는 요인으로 CDA 검증

1. Age_ 1921부터 2023
- EDC_SBA = 심결본인부담금: 연속형
- Age = 생년월일-사망월일로 진행하려 했으나 데이터의 일관성이 없었고 DTH의 null값이 대부분이라 출생년도 1921부터 2023년 현재 생존한 환자로 확인 (현재연령- 연속형)
- TOT_PRSC_DD_CNT= 총 처방일수: 연속형 
- disease_year-첫 진료받은 년도: 범주형 가능할듯. 12~21년 (2023년 기준)

In [3]:
condition = (df_BM['BTH_YYYY'] != '1921LE')
df_BMA = df_BM[condition]
df_BMA

,Unnamed: 0,EDC_SBA,BTH_YYYY,DTH_YYYYMM,MCARE_SUBJ_CD,OPRTN_YN,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,STD_YYYY
11,11,8580,1926,201808.0,1,0,0,1.0,2006
12,12,2900,1926,201808.0,1,0,30,1.0,2008
13,13,2900,1926,201808.0,1,0,30,1.0,2008
14,14,2900,1926,201808.0,1,0,14,1.0,2008
15,15,3500,1926,201808.0,1,0,30,1.0,2008
...,...,...,...,...,...,...,...,...,...
4624,4624,3000,2005,NaN,1,0,30,1.0,2005
4625,4625,3000,2005,NaN,1,0,30,1.0,2005
4626,4626,3000,2005,NaN,1,0,30,1.0,2005
4627,4627,3000,2005,NaN,1,0,30,1.0,2006


In [4]:
df_BMA['BTH_Y']=pd.to_datetime(df_BMA['BTH_YYYY'].astype(str))

C:\Users\01-15\AppData\Local\Temp\ipykernel_9812\1629062973.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_BMA['BTH_Y']=pd.to_datetime(df_BMA['BTH_YYYY'].astype(str))


In [5]:
def present(birth_year):
    current_year = pd.to_datetime('today').year  # 현재 연도 가져오기
    return current_year - birth_year.year 
## chatGPT에게 도움을 받음. 

In [6]:
df_BMA['Age']=df_BMA['BTH_Y'].apply(present) ## 2023년 기준 나이로 계산

C:\Users\01-15\AppData\Local\Temp\ipykernel_9812\1676463516.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_BMA['Age']=df_BMA['BTH_Y'].apply(present) ## 2023년 기준 나이로 계산


In [7]:
## 현재 기준으로 진료 받은 년차
def disease(year) :
    result = 2023 - year
    return result

In [8]:
df_BMA['disease_year']= df_BMA['STD_YYYY'].apply(disease) ## 현재 기준으로 진료 받은 년차

C:\Users\01-15\AppData\Local\Temp\ipykernel_9812\585948669.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_BMA['disease_year']= df_BMA['STD_YYYY'].apply(disease) ## 현재 기준으로 진료 받은 년차


In [9]:
df_BMA[:2]

,Unnamed: 0,EDC_SBA,BTH_YYYY,DTH_YYYYMM,MCARE_SUBJ_CD,OPRTN_YN,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,STD_YYYY,BTH_Y,Age,disease_year
11,11,8580,1926,201808.0,1,0,0,1.0,2006,1926-01-01,97,17
12,12,2900,1926,201808.0,1,0,30,1.0,2008,1926-01-01,97,15


In [10]:
df_BMA.shape

(4618, 12)

In [11]:
df_BMA

,Unnamed: 0,EDC_SBA,BTH_YYYY,DTH_YYYYMM,MCARE_SUBJ_CD,OPRTN_YN,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,STD_YYYY,BTH_Y,Age,disease_year
11,11,8580,1926,201808.0,1,0,0,1.0,2006,1926-01-01,97,17
12,12,2900,1926,201808.0,1,0,30,1.0,2008,1926-01-01,97,15
13,13,2900,1926,201808.0,1,0,30,1.0,2008,1926-01-01,97,15
14,14,2900,1926,201808.0,1,0,14,1.0,2008,1926-01-01,97,15
15,15,3500,1926,201808.0,1,0,30,1.0,2008,1926-01-01,97,15
...,...,...,...,...,...,...,...,...,...,...,...,...
4624,4624,3000,2005,NaN,1,0,30,1.0,2005,2005-01-01,18,18
4625,4625,3000,2005,NaN,1,0,30,1.0,2005,2005-01-01,18,18
4626,4626,3000,2005,NaN,1,0,30,1.0,2005,2005-01-01,18,18
4627,4627,3000,2005,NaN,1,0,30,1.0,2006,2005-01-01,18,17


In [12]:
df_BMA=df_BMA.drop(columns=['DTH_YYYYMM'])

In [13]:
df_BMA

,Unnamed: 0,EDC_SBA,BTH_YYYY,MCARE_SUBJ_CD,OPRTN_YN,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,STD_YYYY,BTH_Y,Age,disease_year
11,11,8580,1926,1,0,0,1.0,2006,1926-01-01,97,17
12,12,2900,1926,1,0,30,1.0,2008,1926-01-01,97,15
13,13,2900,1926,1,0,30,1.0,2008,1926-01-01,97,15
14,14,2900,1926,1,0,14,1.0,2008,1926-01-01,97,15
15,15,3500,1926,1,0,30,1.0,2008,1926-01-01,97,15
...,...,...,...,...,...,...,...,...,...,...,...
4624,4624,3000,2005,1,0,30,1.0,2005,2005-01-01,18,18
4625,4625,3000,2005,1,0,30,1.0,2005,2005-01-01,18,18
4626,4626,3000,2005,1,0,30,1.0,2005,2005-01-01,18,18
4627,4627,3000,2005,1,0,30,1.0,2006,2005-01-01,18,17


In [14]:
df_BMA.shape

(4618, 11)

### 심결본인부담금은?

- ShapiroResult(statistic=0.1199987530708313, pvalue=0.0)
- 심결본인부담금은 비정규분포

In [16]:
stats.shapiro(df_BMA['EDC_SBA'])

ShapiroResult(statistic=0.1199987530708313, pvalue=0.0)

### 연령-심결본인부담금 CDA

- 연령은 심결본인부담금에 영향을 미칠것인가?
- 연속-연속 
    - 귀무 가설: 연령은 심결본인부담금에 영향이 없음.
    - 대립 가설: 연령은 심결본인부담금에 영향이 있음.
    - 분석 결론:
    - ⇒ stats.spearmanr(df_BMA['EDC_SBA'], df_BMA['Age'])
          - SignificanceResult(statistic=0.010781511200300737, pvalue=0.4638696070878372)
      - 귀무 참. p-value > 0.05
      - 연령은 심결본인부담금에 영향이 없다. 

In [11]:
df_BMA=df_BMA.dropna()

In [19]:
df_BMA[['EDC_SBA', 'Age']].corr()

,EDC_SBA,Age
EDC_SBA,1.000000,-0.011429
Age,-0.011429,1.000000


In [20]:
stats.spearmanr(df_BMA['EDC_SBA'], df_BMA['Age'])

SignificanceResult(statistic=0.010781511200300737, pvalue=0.4638696070878372)

### 총 처방일수-심결본인부담금 CDA

- 총 처방일수는 심결본인부담금에 영향을 미칠것인가?
- 연속-연속 
    - 귀무 가설: 총 처방일수는 심결본인부담금에 영향이 없음.
    - 대립 가설: 총 처방일수는 심결본인부담금에 영향이 있음.
    - 분석 결론:
    - ⇒ stats.spearmanr(df_BMA['EDC_SBA'], df_BMA['TOT_PRSC_DD_CNT'])
          - SignificanceResult(statistic=0.0646130250662275, pvalue=1.1116221489820486e-05)
      - 대립이 참. p-value < 0.05
      - 총 처방일수는 심결본인부담금에 영향이 있다
      
      
**변수를 잘못 잡음. 당연히 총 처방일수가 증가할수록 심결본인부담금이 증가할 것으로 예상됨.**

In [22]:
stats.spearmanr(df_BMA['EDC_SBA'], df_BMA['TOT_PRSC_DD_CNT'])

SignificanceResult(statistic=0.0646130250662275, pvalue=1.1116221489820486e-05)

### 질병 연차 - 심결본인부담금 CDA

- 질병 연차는 심결본인부담금에 영향을 미칠것인가?
- 범주-연속 
    - 귀무 가설: 질병 연차는 심결본인부담금에 영향이 없음.
    - 대립 가설: 질병 연차는 심결본인부담금에 영향이 있음.
    - 분석 결론:
    - ⇒ stats.kruskal(df_BMA_12['disease_year'], df_BMA_13['disease_year'], df_BMA_14['disease_year'], df_BMA_15['disease_year'], df_BMA_16['disease_year'], df_BMA_17['disease_year'], df_BMA_18['disease_year'], df_BMA_19['disease_year'], df_BMA_20['disease_year'], df_BMA_21['disease_year'])
          - KruskalResult(statistic=4616.999999999999, pvalue=0.0)
      - 대립이 참. p-value < 0.05
      - 질병 연차는 심결본인부담금에 영향이 있다

In [27]:
df_BMA_drop = df_BMA.drop(columns=['BTH_Y'])

In [28]:
df_BMA_year=pd.pivot_table(data=df_BMA_drop, index='disease_year', aggfunc=sum)
df_BMA_year['year_Disease']=df_BMA_year.index

C:\Users\01-15\AppData\Local\Temp\ipykernel_9812\1074526812.py:1: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_BMA_year=pd.pivot_table(data=df_BMA_drop, index='disease_year', aggfunc=sum)


In [29]:
df_BMA_year

,Age,BTH_YYYY,EDC_SBA,MCARE_RSLT_TYPE,MCARE_SUBJ_CD,OPRTN_YN,STD_YYYY,TOT_PRSC_DD_CNT,Unnamed: 0,year_Disease
disease_year,,,,,,,,,,
12,1041,1933193819381938193819381938196019601960197119...,134100,10.0,2,0,36198,285,37150,12
13,42682,1926192619261926192619261926192619281928192819...,4114690,792.0,1513,9,1547700,25281,1690550,13
14,39256,1926192619261926192619261926192619261926192619...,2944570,743.0,1494,0,1440453,23327,1602939,14
15,37233,1926192619261926192619281928192819281928192819...,3417850,774.0,1334,0,1427688,22782,1677981,15
16,29389,1928192819281928192819281928192819281928192819...,2389000,679.0,853,0,1156032,18750,1403029,16
17,24616,1926192819281928192819281928192819281928192819...,2418950,482.0,875,9,964886,16075,1167713,17
18,22004,1928192819281932193419351935193519351935193519...,1655280,527.0,605,0,848115,13973,1007959,18
19,21827,1928192819281928192819281928192819281932193219...,2024550,1692.0,743,0,811620,12459,921593,19
20,15402,1928193219321932193219381938193819381938193819...,1281580,1237.0,869,9,588882,9664,698427,20


In [32]:
df_BMA_year['year_Disease'].value_counts()

year_Disease
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    1
21    1
Name: count, dtype: int64

In [24]:
df_BMA_12 = df_BMA[(df_BMA['disease_year'] == 12)]
df_BMA_13 = df_BMA[(df_BMA['disease_year'] == 13)]
df_BMA_14 = df_BMA[(df_BMA['disease_year'] == 14)]
df_BMA_15 = df_BMA[(df_BMA['disease_year'] == 15)]
df_BMA_16 = df_BMA[(df_BMA['disease_year'] == 16)]
df_BMA_17 = df_BMA[(df_BMA['disease_year'] == 17)]
df_BMA_18 = df_BMA[(df_BMA['disease_year'] == 18)]
df_BMA_19 = df_BMA[(df_BMA['disease_year'] == 19)]
df_BMA_20 = df_BMA[(df_BMA['disease_year'] == 20)]
df_BMA_21 = df_BMA[(df_BMA['disease_year'] == 21)]

In [25]:
stats.kruskal(df_BMA_12['disease_year'], df_BMA_13['disease_year'], df_BMA_14['disease_year'], df_BMA_15['disease_year'], df_BMA_16['disease_year'], df_BMA_17['disease_year'], df_BMA_18['disease_year'], df_BMA_19['disease_year'], df_BMA_20['disease_year'], df_BMA_21['disease_year'])

KruskalResult(statistic=4616.999999999999, pvalue=0.0)

In [34]:
!pip install prince

                                              0.0/415.6 kB ? eta -:--:--
     -----------                            122.9/415.6 kB 3.6 MB/s eta 0:00:01
     -------------------------------------  409.6/415.6 kB 5.1 MB/s eta 0:00:01
     -------------------------------------- 415.6/415.6 kB 5.2 MB/s eta 0:00:00
                                              0.0/516.2 kB ? eta -:--:--
     ----------                             143.4/516.2 kB 4.3 MB/s eta 0:00:01
     ------------------------------------   491.5/516.2 kB 6.2 MB/s eta 0:00:01
     -------------------------------------- 516.2/516.2 kB 5.4 MB/s eta 0:00:00


In [35]:
import pandas as pd
import prince

In [39]:
# 데이터 프레임 생성
data = df_BMA_year  # 데이터를 적절하게 대체

# CDA를 위해 데이터를 적절하게 가공
cda_data = pd.get_dummies(data, columns=['year_Disease'])

# CDA 모델 생성
cda = prince.CA(n_components=2)  # 원하는 주성분 개수로 설정

# CDA 수행
cda = cda.fit(cda_data)

# CDA 결과 시각화
cda.plot_coordinates(cda_data)

C:\Users\01-15\anaconda3\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\01-15\anaconda3\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\01-15\anaconda3\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


ValueError: Input contains infinity or a value too large for dtype('float64').